# llama factory

## 基本信息
一个支持跨平台的综合大模型训练框架。测试下来，目前是最顺手的方式，唯一的美中不足是在mps的环境下，不支持qlora量化训练。

特点：
- 多模型，基本知名的模型都有了
- 多方式调试：预训练和sft，lora等
- 支持多数据集和自定义数据集
- 支持自动匹配提示模版
- 开箱即用
- 支持调用mps运算

## 任务场景

### 安装
```shell
git clone https://github.com/hiyouga/LLaMA-Factory.git
conda create -n llama_factory python=3.10
conda activate llama_factory
cd LLaMA-Factory
pip install -r requirements.txt
```

### 启动
```shell
python src/train_web.py
```
会自动打开浏览器访问：http://localhost:7860

下载hf上的资源需要代理，可以直接用MODELSCOPE:
```shell
USE_MODELSCOPE_HUB=1 python src/train_web.py
```

### 添加自定义数据集
比如我们可以从`identity`开始；

#### 新增数据集
```shell
cp identity.json identity_fofabot.json
sed -i '' 's/NAME/FOFABot/g' identity_fofabot.json
sed -i '' 's/AUTHOR/HuaShunXinAn/g' identity_fofabot.json
```
#### 修改`data/dataset_info.json`文件，新增一个对象
```json
{
  "identity": {
    "file_name": "identity.json",
    "file_sha1": "ffe3ecb58ab642da33fbb514d5e6188f1469ad40"
  },
  "identity_fofabot": {
    "file_name": "identity_fofabot.json"
  }
}
```
其中file_sha1可选，我们去掉。直接刷新前台页面就能够看到了。

## 常见问题

## 实验
### 能否正确加载jsonl文件？
现象：jsonl格式的数据集配置好以后，用“预览数据集”显示不出来，但是训练看起来又没有问题。

自带的数据格式为一个完整的json，需要取跟踪一下底层实现。

观察到底层就是直接调用了[load_dataset](./datasets.ipynb)

预览数据库的时候get_preview是根据后缀来判断格式的，如果是json代表是完整的一个对象，如果是jsonl才代表一行一个对象。
```python
def get_preview(dataset_dir: str, dataset: list, page_index: int) -> Tuple[int, list, Dict[str, Any]]:
    with open(os.path.join(dataset_dir, DATA_CONFIG), "r", encoding="utf-8") as f:
        dataset_info = json.load(f)

    data_file: str = dataset_info[dataset[0]]["file_name"]
    with open(os.path.join(dataset_dir, data_file), "r", encoding="utf-8") as f:
        if data_file.endswith(".json"):
            data = json.load(f)
        elif data_file.endswith(".jsonl"):
            data = [json.loads(line) for line in f]
        else:
            data = [line for line in f]  # noqa: C416
    return len(data), data[PAGE_SIZE * page_index : PAGE_SIZE * (page_index + 1)], gr.update(visible=True)
```
所以只是预览的问题，训练的时候不会受影响，因为`load_dataset('json', data_files=[])`的方式会自动进行处理。关于load_dataset对json的处理可以参考[datasets中加载json和jsonl的区别](./datasets.ipynb)


### instraction到训练数据的转换
默认数据是alpaca的json格式，llama-factory里面有模版template，是如何进行转换的呢？

模版通过template参数指定，比如yi模型，对应的代码是：
```python
_register_template(
    name="yi",
    format_user=StringFormatter(slots=["<|im_start|>user\n{{content}}<|im_end|>\n<|im_start|>assistant\n"]),
    format_separator=EmptyFormatter(slots=["\n"]),
    stop_words=["<|im_end|>"],
    replace_eos=True,
)
```
转换的调用堆栈：
```
apply, formatter.py:102
_encode, template.py:92
encode_multiturn, template.py:65
preprocess_supervised_dataset, preprocess.py:57
apply_function_on_filtered_inputs, arrow_dataset.py:3361
_map_single, arrow_dataset.py:3482
map, arrow_dataset.py:3105
wrapper, arrow_dataset.py:558
wrapper, arrow_dataset.py:593
get_dataset, loader.py:178
run_sft, workflow.py:32
run_exp, tuner.py:31
main, train_bash.py:5
<module>, train_bash.py:14
```
这里就是替换变量的过程了，到formatter里面来的时候，content变量是"{instruction}\n{input}"格式，这个是在`convert_alpaca`函数里面转换的：
```python
content = []
if dataset_attr.prompt and examples[dataset_attr.prompt][i]:
    content.append(examples[dataset_attr.prompt][i])

if dataset_attr.query and examples[dataset_attr.query][i]:
    content.append(examples[dataset_attr.query][i])

prompt.append({"role": Role.USER.value, "content": "\n".join(content)})
```
调用堆栈：
```
convert_alpaca, aligner.py:32
apply_function_on_filtered_inputs, arrow_dataset.py:3361
_map_single, arrow_dataset.py:3482
map, arrow_dataset.py:3105
wrapper, arrow_dataset.py:558
wrapper, arrow_dataset.py:593
align_dataset, aligner.py:127
load_single_dataset, loader.py:111
get_dataset, loader.py:162
run_sft, workflow.py:32
run_exp, tuner.py:31
main, train_bash.py:5
<module>, train_bash.py:14
```

总结一下，加入输入为`{"instruction":"回答如下关于fofa基本规则的说明。", "input":"fofa网址是？","output":"fofa网址是：https://fofa.info"}`，那么流程：
- iio的格式转换为prst的格式，生成：`{'prompt': [[{'content': '回答如下关于fofa基本规则的说明。\nfofa网址是？', 'role': 'user'}]], 'response': [[{'content': 'fofa网址是：https://fofa.info', 'role': 'assistant'}]], 'system': [''], 'tools': ['']}`
- prst的格式生成模版的格式，生成：`<|im_start|>user\n回答如下关于fofa基本规则的说明。\nfofa网址是？<|im_end|>\n<|im_start|>assistant\n`


### 外部工具调用
模型选择`DeepSeekCoder-7B-Chat`（base模型也行，测试通过），数据集选择`glaive_toolcall`，其他参数默认。
训练一天后差不多了，loss大约0.3-0.5之间徘徊，可以测试：
- 加载模型
- 输入写上：`what's the weather like in San Francisco today`
- 工具列表构造一个：
```json
[
    {
      "name": "get_current_weather",
      "description": "Get the current weather",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city, e.g. San Francisco"
          }
        },
        "required": ["location"]
      }
    }
]
```

这样就能看到返回格式变成了：
```
Action: get_current_weather
Action Input: {"location": "San Francisco"}
```

#### 要多少数据？
参考[trelis讲function-calling](https://trelis.com/function-calling/)，里面提到：
- 数据量只要几十条就好，要包含有调用的，和没有调用的
- 另外也要进行chat_template的修改，https://huggingface.co/Trelis/Qwen1.5-function-calling-chat-template/blob/main/tokenizer_config.json
```
"chat_template": "{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}<|im_start|>system\nYou are a helpful assistant<|im_end|>\n{% endif %}{% if message['role'] == 'function_metadata' %}<|im_start|>tools\nYou have access to the following functions. Use them only if required:\n\n{{ message['content'] }}\n\nTo call a function, respond with a JSON object in this format:\n{\n    \"name\": \"function_name\",\n    \"arguments\": {\n        \"argument1\": \"value1\",\n        \"argument2\": \"value2\"\n    }\n}\nRespond with a JSON object only if you wish to make a function call. Any other response will be treated as a regular query.When making a function call, provide only the JSON object, nothing else. Make one function call at a time. After the function call, wait for the response.\nOnly make use of the functions if they assist in providing the user with an answer. Otherwise, answer without making a function call.{% elif message['role'] == 'function_response' %}<|im_start|>function_response\nHere is the response to the function call. If helpful, use it to respond to the user's question:{{ message['content'] }}{% else %}{{'<|im_start|>' + message['role'] + '\n' + message['content']}}{% endif %}{% if (loop.last and add_generation_prompt) or not loop.last %}{{ '<|im_end|>' + '\n'}}{% endif %}{% endfor %}{% if add_generation_prompt and messages[-1]['role'] != 'assistant' %}{{ '<|im_start|>assistant\n' }}{% endif %}",
```